## Computational Theory Assessment

In [ ]:
import numpy as np

## Problem 1: Binary Words and Operations

## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords

## END